In [13]:
# ============================================================================
# STEP 1: Install Required Libraries
# ============================================================================
print("Step 1: Installing required libraries...")
import sys
import subprocess

# Install groq, pillow, and gradio
subprocess.check_call([sys.executable, "-m", "pip", "install", "groq", "pillow", "gradio", "-q"])
print("✅ Libraries installed successfully!")

Step 1: Installing required libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.6 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


✅ Libraries installed successfully!


In [14]:
# ============================================================================
# STEP 2: Import Libraries
# ============================================================================
print("\nStep 2: Importing libraries...")
import time
time.sleep(1)  # Give a moment for installation to complete

from groq import Groq
import base64
from PIL import Image
import gradio as gr
import io


Step 2: Importing libraries...


In [15]:
# ============================================================================
# STEP 3: Set Up Groq API Key
# ============================================================================
print("\nStep 3: Setting up Groq API...")
API_KEY = "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"
client = Groq(api_key=API_KEY)


Step 3: Setting up Groq API...


In [16]:
# ============================================================================
# STEP 4: Helper Functions
# ============================================================================
print("\nStep 4: Creating helper functions...")

def encode_image_to_base64(image):
    """Convert PIL Image to base64 data URL"""
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    data_url = f"data:image/png;base64,{img_str}"
    return data_url

def analyze_graph(image, question, history):
    """Analyze the graph and return response"""
    if image is None:
        return history + [("Please upload an image first.", None)]
    
    if not question.strip():
        return history + [("Please ask a question about the graph.", None)]
    
    try:
        # Convert image to base64
        image_url = encode_image_to_base64(image)
        
        # Call Groq API
        completion = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": image_url
                            }
                        },
                        {
                            "type": "text",
                            "text": question
                        }
                    ]
                }
            ],
            temperature=0.7,
            max_tokens=2048,
            top_p=1,
            stream=False
        )
        
        response = completion.choices[0].message.content
        history = history + [(question, response)]
        return history
    
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        history = history + [(question, error_msg)]
        return history


Step 4: Creating helper functions...


In [ ]:
# ============================================================================
# STEP 5: Gradio Interface
# ============================================================================
print("\nStep 5: Creating Gradio interface...")

with gr.Blocks(title="AI Graph Analyzer", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🤖 AI Graph Analyzer Chatbot
        ### Powered by Groq & Llama Vision
        
        Upload a graph, chart, or diagram and ask questions about it!
        """
    )
    
    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(
                type="pil",
                label="Upload Your Graph/Chart",
                height=400
            )
            gr.Markdown("**Examples:** Line graphs, bar charts, pie charts, scatter plots, etc.")
        
        with gr.Column(scale=1):
            chatbot = gr.Chatbot(
                label="Conversation",
                height=400,
                show_label=True
            )
            
            question_input = gr.Textbox(
                placeholder="Ask a question about the graph...",
                label="Your Question",
                lines=2
            )
            
            with gr.Row():
                submit_btn = gr.Button("🔍 Analyze", variant="primary")
                clear_btn = gr.Button("🗑️ Clear Chat")
    
    gr.Markdown(
        """
        ### Example Questions:
        - What is the overall trend shown in this graph?
        - What are the highest and lowest values?
        - Compare the values between different categories
        - What insights can you derive from this chart?
        - Explain what this diagram represents
        """
    )
    
    # Event handlers
    submit_btn.click(
        fn=analyze_graph,
        inputs=[image_input, question_input, chatbot],
        outputs=chatbot
    ).then(
        lambda: "",
        outputs=question_input
    )
    
    question_input.submit(
        fn=analyze_graph,
        inputs=[image_input, question_input, chatbot],
        outputs=chatbot
    ).then(
        lambda: "",
        outputs=question_input
    )
    
    clear_btn.click(
        lambda: [],
        outputs=chatbot
    )

# Launch the interface
print("\n" + "="*60)
print("🚀 Launching Gradio interface...")
print("="*60)

demo.launch(share=True, debug=True)


Step 5: Creating Gradio interface...

🚀 Launching Gradio interface...


/tmp/ipykernel_47/1376310908.py:26: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://cf18fe231484846838.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
